In [9]:
# url을 가져와서 문서로 만드는 작업
import requests
from bs4 import BeautifulSoup as bs

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

In [1]:
from google.colab import drive
drive.mount('/content/mnt')

Mounted at /content/mnt


In [28]:
filepath = '/content/mnt/MyDrive/proj1_1조/data/기상청/'

- 일자, 타임별 체감온도, 불쾌지수, 우산 유무

In [4]:
# sensible temperature 체감온도
# discomfort index 불쾌지수
# df를 변수로 받아서 3시간 단위로 주어지는 불쾌지수와 체감온도의 평균을 계산하고
# 강수량과 적설량을 계산하여 우산을 써야하는지 판단
# 새로운 DataFrame으로 만들어서 반환하는 함수

def make_st_di_um(df):
  temp_list=[]
  # 순서i 와 idx를 따로 받아서 iloc를 해서 호출해야 정상적으로 불러지고,
  # 날짜 데이터를 가져올때는 idx
  for i, idx in enumerate(lun_df.index):
    if idx % 3 == 0: # 첫 시작 idx를 찾는다
      temp = lun_df.iloc[i:i+3] # 3개의 row를 가져와 계산

      umb = 0 # 우산유무
      for r in temp['강수량'].values:
        if r > 0.0:
          umb = 1
        
      if umb == 0:
        for s in temp['적설량'].values:
          if s > 0.0:
            umb = 1

      tem_mean = temp['체감온도'].mean()
      di_mean = temp['불쾌지수'].mean()
        
      # temp['날짜'][idx] 로 호출해야 정상적으로 그 묶음의 날짜를 가져올 수 있음
      temp_list.append({'날짜': temp['날짜'][idx], '체감온도': np.round(tem_mean,1), '불쾌지수': int(di_mean), '우산': umb})
  new_df = pd.DataFrame(temp_list, columns = ['날짜', '체감온도', '불쾌지수', '우산'])
  return new_df

## 기상청 api 연결

In [5]:
encoding_k = '5hKnz%2FSOwgTelwTnXx36H%2BKOB9ObVv9LZ5KPCTZiYY0hYEubhGfovemBVL884JkVyGPNNj5aD%2BYPJ2k3cKFkvg%3D%3D'

In [30]:
start_date = '20210310' # train 데이터의 시작 일자
end_date = '20210409' # train 데이터의 마감 일자
page_no = '1'
num_rows = '999' # 최대 1000개 이상을 요청할수는 없음..
location = '192' # 진주 위치코드

In [31]:
baseurl = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=' + encoding_k
url1 = '&numOfRows=' + num_rows
url2 = '&pageNo=' + page_no
url3 = '&dataCd=ASOS&dateCd=HR'
url4 = '&stnIds=' + location # 위치 코드
url5 = '&endDt=' + end_date
url6 = '&endHh=' + '19' #
url7 = '&startHh='+ '11' # 1시간 단위로 체크
url8 = '&startDt=' + start_date

url = baseurl + url1 + url2 + url3 + url4 + url5 + url6 + url7 + url8
print(url)

http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=5hKnz%2FSOwgTelwTnXx36H%2BKOB9ObVv9LZ5KPCTZiYY0hYEubhGfovemBVL884JkVyGPNNj5aD%2BYPJ2k3cKFkvg%3D%3D&numOfRows=999&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=192&endDt=20210409&endHh=19&startHh=11&startDt=20210310


In [32]:
xml = requests.get(url)
soup = bs(xml.text, 'html.parser')

In [ ]:
soup

In [33]:
# - tm : 시간
# - ta : 기온
# - rn : 강수량
# - ws : 풍속
# - hm : 습도
# - dsnw : 적설
# - ts : 지면온도

# 필요한 내용만 따로 저장
data = soup.find_all('item')
con_lst=[]

for item in data:
    time = item.find('tm').text # 시간
    temper = item.find('ta').text # 기온
    rain = item.find('rn').text # 강수량
    wind = item.find('ws').text # 풍속
    hm = item.find('hm').text # 습도
    snow = item.find('dsnw').text # 적설
    tem_s = item.find('ts').text # 지면온도
    con_lst.append({'날짜':time, '기온':temper, '습도':hm, 
                    '강수량':rain, '적설량':snow, '풍속':wind})
# dataframe 생성
df = pd.DataFrame(con_lst, columns = ['날짜','기온','습도','강수량','적설량','풍속'])

In [34]:
# '' 빈 값을 nan값으로 바꾼후 결측치 제거
df.강수량.replace('', np.nan, inplace=True)
df.적설량.replace('', np.nan, inplace=True)

df['강수량'] = df['강수량'].fillna(0)
df['적설량'] = df['적설량'].fillna(0)

In [35]:
df.head()

,날짜,기온,습도,강수량,적설량,풍속
0,2021-03-10 11:00,13.4,41,0,0.0,1.9
1,2021-03-10 12:00,14.7,38,0,0.0,2.6
2,2021-03-10 13:00,15.6,36,0,0.0,3.1
3,2021-03-10 14:00,16.6,34,0,0.0,1.8
4,2021-03-10 15:00,17.4,30,0,0.0,1.4


In [47]:
# 결측치 제거 판단 후에 진행함. 
# 2016.06.27 20:00, 21:00에 기온 풍속 습도 데이터가 '' 빈값이었으나 이후 계산에는 영향을 미치지 않을것이라 판단 0으로 치환
# 2018.01.13 14:00 기온 풍속 습도 데이터가 '' 빈값, 위와 마찬가지로 계산엔 영향없는 시간대
# 2018.02.19 20:00
# 2018.02.20 15:00, 16:00
# 2018.02.26 20~23
# 2018.03.11 13~16 13시값은 계산에 영향이 있음.. 
# df['기온'][314] = df['기온'][313] 12시 값으로 대체..
# df['기온'][314]
# 2018.10.19 11~15 11,12,13 모두 영향 <- 이 날의 최고기온으로 대체 최고기온: 20.8도
# 2018.11.01 08~11 11시 영향 <- 12값으로 대체
# df['기온'][600] = df['기온'][601] = df['기온'][602] = 20.8
# df['기온'][912] = df['기온'][913]
# 2021.04.03 17~19 전체가 영향..
# df['기온'][582] = df['기온'][581]
# df['기온'][583] = df['기온'][582]
# df['기온'][584] = df['기온'][583]
# 풍속데이터가 빈 값인 경우.. 찾기가 어려운 상황임..
# '' 빈 값을 nan값으로 바꾼후 결측치 제거

df.기온.replace('', np.nan, inplace=True)
df.풍속.replace('', np.nan, inplace=True)
df.습도.replace('', np.nan, inplace=True)
df['기온'] = df['기온'].fillna(0)
df['풍속'] = df['풍속'].fillna(0)
df['습도'] = df['습도'].fillna(0)

In [48]:
# 형변환
numeric_list = ['기온', '습도', '강수량', '적설량', '풍속']

for x in numeric_list:
  try:
    df[x] = df[x].astype(float)
  except:
    print(x)

In [39]:
df[df['기온'] =='']

,날짜,기온,습도,강수량,적설량,풍속
585,2021-04-03 20:00,,100.0,1.7,0.0,0.0
586,2021-04-03 21:00,,100.0,0.7,0.0,0.0
587,2021-04-03 22:00,,100.0,0.3,0.0,0.0
588,2021-04-03 23:00,,100.0,0.5,0.0,0.1
589,2021-04-04 00:00,,100.0,0.1,0.0,0.0
590,2021-04-04 01:00,,100.0,0.0,0.0,0.0
591,2021-04-04 02:00,,100.0,0.0,0.0,0.4


In [ ]:
df[df['습도'] =='']

,날짜,기온,습도,강수량,적설량,풍속
437,2019-03-29 16:00,,,0.0,0.0,2.2


In [ ]:
df[df['풍속'] =='']

In [49]:
# DI = 9/5Ta-0.55(1-RH)(9/5Ta-26)+32
df['불쾌지수'] = (9/5) * df['기온'] - 0.55 * (1-df['습도']/100) * (((9/5) * df['기온']) - 26) + 32
df['불쾌지수'] = df.불쾌지수.astype(int)

# 체감온도(℃)=13.12+0.6215×T-11.37V^0.16+0.3965V^0.16×T
df['체감온도'] = np.round(13.12 + 0.6215 * df['기온'] - 11.37 * df['풍속'] ** 0.16 + 0.3965 * df['풍속'] ** 0.16 * df['기온'], 1)

In [ ]:
df['시간'] = df['날짜'].apply(lambda x : x.split(' ')[1])
df['날짜'] = df['날짜'].apply(lambda x : x.split(' ')[0])
df['시간'] = df['시간'].apply(lambda x: int(x.split(':')[0])) # 계산의 편의성을 위해 시간데이터를 00~23으로 변환

In [51]:
lun_df = df[(df['시간'] > 10) & (df['시간'] < 14)] # 점심 데이터
eve_df = df[(df['시간'] > 16) & (df['시간'] < 20)] # 저녁 데이터

In [52]:
new_lun_df = make_st_di_um(lun_df)
new_eve_df = make_st_di_um(eve_df)

In [53]:
new_lun_df.tail()

,날짜,체감온도,불쾌지수,우산
26,2021-04-05,14.9,56,0
27,2021-04-06,16.7,57,0
28,2021-04-07,22.4,64,0
29,2021-04-08,21.7,63,0
30,2021-04-09,18.2,60,0


In [54]:
new_eve_df.tail()

,날짜,체감온도,불쾌지수,우산
26,2021-04-05,14.9,56,0
27,2021-04-06,16.7,57,0
28,2021-04-07,22.4,64,0
29,2021-04-08,21.7,63,0
30,2021-04-09,18.2,60,0


In [55]:
prev_lun = pd.read_csv(filepath+'일별_점심시간_0309.csv')
prev_eve = pd.read_csv(filepath+'일별_저녁시간_0309.csv')

- row아래로 합치기

In [56]:
merge_lun = pd.concat([prev_lun, new_lun_df], ignore_index=True)
merge_eve = pd.concat([prev_eve, new_eve_df], ignore_index=True)

In [57]:
merge_lun.tail()

,날짜,체감온도,불쾌지수,우산
1889,2021-04-05,14.9,56,0
1890,2021-04-06,16.7,57,0
1891,2021-04-07,22.4,64,0
1892,2021-04-08,21.7,63,0
1893,2021-04-09,18.2,60,0


In [58]:
merge_eve.tail()

,날짜,체감온도,불쾌지수,우산
1889,2021-04-05,14.9,56,0
1890,2021-04-06,16.7,57,0
1891,2021-04-07,22.4,64,0
1892,2021-04-08,21.7,63,0
1893,2021-04-09,18.2,60,0


- 파일 병합

In [59]:
merge_lun.to_csv(filepath+'일별_점심시간_0409.csv', index=False)
merge_eve.to_csv(filepath+'일별_저녁시간_0409.csv', index=False)